# Data wrangling : sparsify, use indices for labels

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse as sps
%matplotlib widget

In [ ]:
envs = pd.read_csv('training_environments.csv', index_col=0)
empo_names = [f'empo_{i}' for i in range(1, 4)]
empo_index_to_label = []

for empo in empo_names:
    empo_index_to_label.append([str(row) for row in envs.drop_duplicates(subset=empo)[empo]])
    
empo_label_to_index = {name : {label : i for i, label in enumerate(labels)} for name, labels in zip(empo_names, empo_index_to_label)}
empo_label_to_index

In [ ]:
# replace text labels with integers
envs = envs.replace(empo_label_to_index)

In [ ]:
import numpy as np

In [ ]:
def save_as_sparse(in_filename, out_filename):
    line_count = sum(1 for line in open(in_filename))
    rows = []
    with open(in_filename) as f:
        for i, line in enumerate(f):
            if i == 0:
                continue
            row = [int(x) for x in line.strip().split(',')[1:]]
            row = sps.csr_matrix(row)
            rows.append(row)

            if i % 1000 == 0:
                print(f'Sparsifying {in_filename} [row {i} / {line_count}]\r')
    mat = sps.vstack(rows)
    
    sps.save_npz(out_filename, mat)

In [ ]:
from pathlib import Path

def maybe_sparsify(in_filename, out_filename):
    if not Path(out_filename).is_file():
        save_as_sparse(in_filename, out_filename)
        
def get_header_line(csv_file):
    with open(csv_file) as f:
        line = next(f)
        return np.array(line.rstrip().split(',')[1:])
    
maybe_sparsify('training_descriptors.csv', 'training_descriptors_sparse.npz')
maybe_sparsify('challenge_descriptors.csv', 'challenge_descriptors_sparse.npz')

In [ ]:
desc_species_names = get_header_line('training_descriptors_header.csv')

desc = sps.load_npz('training_descriptors_sparse.npz')
species = pd.read_csv('bacterial_species.csv', index_col=0)

In [ ]:
np.random.choice(desc_species_names, 3)

In [ ]:
def sparse_megabytes(a):
    return (a.data.nbytes + a.indptr.nbytes + a.indices.nbytes) / (1024 * 1024)

print(f'In-memory size of desc : {sparse_megabytes(desc):.2f}M')

In [ ]:
def to_taxonomy_df(desc, taxon_level):
    taxons = species[taxon_level][desc_species_names]
    columns = taxons.unique()

    taxon_indices, taxon_names = pd.factorize(taxons)
    
    data = np.ones(taxon_indices.shape)
    row_ind = np.arange(taxon_indices.shape[0])
    col_ind = taxon_indices
    
    D = sps.csr_matrix((data, (row_ind, col_ind)))
    
    table = desc @ D
    
    return taxon_names, table

taxon_names = {}
taxons = {}

for taxon_level in species.columns:
    taxon_names[taxon_level], taxons[taxon_level] = to_taxonomy_df(desc, taxon_level)

# Classification

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_validate

clf_logit = LogisticRegression(random_state=0, n_jobs=-1)
clf_rforest = RandomForestClassifier(random_state=0, n_jobs=-1)

In [ ]:
def cross_validate_clf(clf, desc, empo, samples=None, fast=False, n_splits=5):
    samples = samples if samples is not None else desc.shape[0]
    
    # shuffle and truncate data
    idx = np.arange(samples)

    gen = np.random.default_rng(0)
    gen.shuffle(idx)
    idx = idx[:samples]
    
    desc = desc[idx]
    empo = empo[idx]
    
    if fast:
        print('Warning : using fast evaluation, cross-validation turned off.')
        
        train_pcent = 1 - 1 / n_splits
        train_count = int(round(train_pcent * samples))

        desc_train = desc_shuf[:train_count]
        desc_validate = desc_shuf[train_count:]

        empo_train = empo[:train_count]
        empo_validate = empo[train_count:]

        clf.fit(desc_train, empo_train)
        accuracy = clf.score(desc_validate, empo_validate)
        accuracies = np.array([accuracy])
        
        f1 = f1_score(clf.predict(desc_validate), empo_validate, average='weighted')
        f1 = np.array([f1])
        
        return {'test_accuracy' : accuracies, 'test_f1_weighted' : f1}
    else:
        # cross validation
        k_folds = KFold(n_splits=n_splits, shuffle=True, random_state=0)
    
        return cross_validate(clf, desc, empo, cv=k_folds, scoring=['accuracy', 'f1_weighted'], n_jobs=-1)
    
def train_clf(clf, desc, envs, empo):
    clf.fit(desc, envs[empo])

In [ ]:
scores = []
features = [(taxon_level, taxons[taxon_level]) for taxon_level in species.columns]
features.append(('desc', desc))

samples=None
for clf_name, clf in [('rforest', clf_rforest), ('logit', clf_logit)]:
    for empo in empo_names:
        for feature_name, feature_vector in features:
            print(clf_name, empo, feature_name)
            
            s = cross_validate_clf(clf, feature_vector, envs[empo], samples=samples)
            s['clf_name'] = clf_name
            s['empo'] = empo
            s['features'] = feature_name
        
            scores.append(s)

scores = pd.DataFrame(scores)

In [ ]:
for k in ['test_accuracy', 'score_time', 'test_f1_weighted', 'fit_time']:
    scores[k + '_median'] = scores[k].apply(lambda x : np.median(x))

In [ ]:
scores.to_csv('scores.csv')

In [ ]:
cross_validate_clf(clf_logit, taxons['taxonomy_6'], envs['empo_3'])

In [ ]:
tx_level = 'taxonomy_2'
feature_vector = taxons[tx_level]
n_features = feature_vector.shape[-1]

# D_0 : Domain
# D_1 : ?
# D_2 : Class
# D_3 : Order
# D_4 : Family
# D_5 : ?
# D_6 : ?

clf_logit.fit(feature_vector, envs['empo_1'])

In [ ]:
feature_idx = np.argsort(clf_logit.coef_[0])
plt.plot(range(n_features), sorted(np.log10(abs(clf_logit.coef_[0][feature_idx]))))
plt.show()

n = 50
print(taxon_names[tx_level][feature_idx[-50:]])
print(taxon_names[tx_level][feature_idx[:50]])

In [ ]:
def biological_interpretation(taxon_level, empo_name):
    feature_vector = taxons[taxon_level]
    n_features = feature_vector.shape[-1]

    # D_0 : Domain
    # D_1 : ?
    # D_2 : Class
    # D_3 : Order
    # D_4 : Family
    # D_5 : ?
    # D_6 : ?

    clf_logit.fit(feature_vector, envs[empo_name])
    feature_order_idx = np.argsort(clf_logit.coef_, axis=0)
    
    print(feature_order_idx.shape)
    print(taxon_names[taxon_level][feature_order_idx[-50:]])
    print(taxon_names[taxon_level][feature_order_idx[:50]])
    
biological_interpretation('taxonomy_2', 'empo_2')

# Dimensionality reduction

In [ ]:
print(desc)
for taxonomy in list(species):
    print(taxonomy, species[taxonomy].unique().size)

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=200)
svd.fit(desc)

In [ ]:
svd.explained_variance_ratio_.sum()

In [ ]:
desc_reduced = svd.transform(desc)

In [ ]:
cross_validate_clf(clf_logit, desc_reduced, envs, 'empo_1')

In [ ]:
cross_validate_clf(clf_logit, desc_reduced, envs, 'empo_2')

In [ ]:
cross_validate_clf(clf_logit, desc_reduced, envs, 'empo_3')

In [ ]:
svd.components_.shape

In [ ]:
plt.ylim(0, 1)
plt.plot(svd.explained_variance_ratio_.cumsum())

In [ ]:
plt.imshow(svd.components_ > 1e-6, aspect='auto')
plt.colorbar()

In [ ]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_classif
selector = SelectKBest(mutual_info_classif, k=200)

In [ ]:
n=100
selector.fit(desc[:n], envs['empo_1'][:n])

# Challenge prediction

In [ ]:
challenge_desc = sps.load_npz('challenge_descriptors_sparse.npz')

In [ ]:
train_clf(clf_logit, desc, envs, 'empo_1')